# Set-up

## Imports

In [ ]:
import corr_utils.covariate as utils
import corr_utils.analysis as analysis_utils

In [ ]:
import pandas as pd
import pandas as pd
import numpy as np

In [ ]:
from sklearn.model_selection import GridSearchCV
from stepmix import StepMix
from stepmix.bootstrap import blrt_sweep

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import six

In [ ]:
from importlib import reload
reload(utils)
reload(analysis_utils)

## Configurations

In [78]:
grid = {
    'n_components': [1, 2, 3, 4, 5, 6], # number of latent classes
    'n_steps' : [1] # number of steps in the estimation (see: https://stepmix.readthedocs.io/en/latest/api.html#stepmix)
}

## Utils

In [79]:
def render_mpl_table(data, col_width=3.0, row_height=0.625, font_size=14,
                     header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
                     bbox=[0, 0, 1, 1], header_columns=0,
                     ax=None, **kwargs):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, **kwargs)

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in  six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])
    return ax

# reference: https://stackoverflow.com/a/39358722

# Data Preparation

## Acquisistion

In [80]:
# define columns needed

base_columns = ['case_id', 'age_during_op', 'female_sex']

elixhauser_score_columns = ['elixhauser_van_walraven']

original_elixhauser_category_columns = ['aids/hiv_elixhauser', 'alcohol_abuse_elixhauser',
'blood_loss_anemia_elixhauser', 'cardiac_arrhythmias_elixhauser',
'chronic_pulmonary_disease_elixhauser', 'coagulopathy_elixhauser',
'congestive_heart_failure_elixhauser', 'deficiency_anemia_elixhauser',
'depression_elixhauser', 'diabetes_complicated_elixhauser',
'diabetes_uncomplicated_elixhauser', 'drug_abuse_elixhauser',
'fluid_and_electrolyte_disorders_elixhauser',
'hypertension_complicated_elixhauser',
'hypertension_uncomplicated_elixhauser', 'hypothyroidism_elixhauser',
'liver_disease_elixhauser', 'lymphoma_elixhauser',
'metastatic_cancer_elixhauser', 'obesity_elixhauser',
'other_neurological_disorders_elixhauser', 'paralysis_elixhauser',
'peptic_ulcer_disease_excluding_bleeding_elixhauser',
'peripheral_vascular_disorders_elixhauser', 'psychoses_elixhauser',
'pulmonary_circulation_disorders_elixhauser',
'renal_failure_elixhauser',
'rheumatoid_arthritis/collagen_vascular_diseases_elixhauser',
'solid_tumor_without_metastasis_elixhauser',
'valvular_disease_elixhauser', 'weight_loss_elixhauser']

elixhauser_outcome_column = ['in_hospital_death']

exclusion_column = ['asa_status']

In [81]:
# get (final) study cohort
df_elixhauser = pd.read_csv('data/subphenotypes/subphenotypes_cohort_data.csv', delimiter = ',', usecols = base_columns + elixhauser_score_columns + original_elixhauser_category_columns + elixhauser_outcome_column + exclusion_column)

## Cleaning

### Names

In [88]:
def clean_names(df_original:pd.DataFrame):

    df = df_original.copy()

    rename_dict = {col: col.replace('_elixhauser', '').replace('_', ' ') for col in original_elixhauser_category_columns}
    df = df.rename(columns=rename_dict)
    elixhauser_category_columns_cleaned = [col.replace('_elixhauser', '').replace('_', ' ') for col in original_elixhauser_category_columns]

    rename_dict = {
        'aids/hiv': 'HIV Kat. C',
        'alcohol abuse': 'Alkoholabusus',
        'blood loss anemia': 'Blutungsanämie',
        'cardiac arrhythmias': 'HRST',
        'chronic pulmonary disease': 'ChronLungenE',
        'coagulopathy': 'Gerinnungsstörungen',
        'congestive heart failure': 'CHF',
        'deficiency anemia': 'Mangelanämie',
        'depression': 'Depression',
        'diabetes complicated': 'DM, kompliziert',
        'diabetes uncomplicated': 'DM, unkompliziert',
        'drug abuse': 'Drogenabusus',
        'fluid and electrolyte disorders': 'Elytstörung',
        'hypertension complicated': 'Hypertonus, kompliziert',
        'hypertension uncomplicated': 'Hypertonus, unkompliziert',
        'hypothyroidism': 'Hypothyr.',
        'liver disease': 'LeberE',
        'lymphoma': 'Lymphom',
        'metastatic cancer': 'Metastasen',
        'obesity': 'Adipositas',
        'other neurological disorders': 'Neuro, andere',
        'paralysis': 'Lähmung',
        'peptic ulcer disease excluding bleeding': 'PU',
        'peripheral vascular disorders': 'pAVK',
        'psychoses': 'Psychosen',
        'pulmonary circulation disorders': 'PulmCircE',
        'renal failure': 'Niereninsuffizienz',
        'rheumatoid arthritis/collagen vascular diseases': 'RA/KVE',
        'solid tumor without metastasis': 'M0',
        'valvular disease': 'HKE',
        'weight loss': 'Gewichtsverlust'
    } 

    rename_dict_english = {
    'aids/hiv': 'HIV Cat. C',
    'alcohol abuse': 'AlcoholAbuse',
    'blood loss anemia': 'AnemiaBL',
    'cardiac arrhythmias': 'Arrhythmia',
    'chronic pulmonary disease': 'ChronicLungDis',
    'coagulopathy': 'Coagulopathy',
    'congestive heart failure': 'CHF',
    'deficiency anemia': 'DefAnemia',
    'depression': 'Depression',
    'diabetes complicated': 'DM, Comp',
    'diabetes uncomplicated': 'DM, Uncomp',
    'drug abuse': 'DrugAbuse',
    'fluid and electrolyte disorders': 'ElectrolyteDis',
    'hypertension complicated': 'HTN, Comp',
    'hypertension uncomplicated': 'HTN, Uncomp',
    'hypothyroidism': 'Hypothyroidism',
    'liver disease': 'LiverDis',
    'lymphoma': 'Lymphoma',
    'metastatic cancer': 'MetastaticCa',
    'obesity': 'Obesity',
    'other neurological disorders': 'OtherNeuro',
    'paralysis': 'Paralysis',
    'peptic ulcer disease excluding bleeding': 'PUD',
    'peripheral vascular disorders': 'PVD',
    'psychoses': 'Psychoses',
    'pulmonary circulation disorders': 'PulmCircDis',
    'renal failure': 'RenalFail',
    'rheumatoid arthritis/collagen vascular diseases': 'RA/CVD',
    'solid tumor without metastasis': 'SolidTumor',
    'valvular disease': 'ValveDis',
    'weight loss': 'WeightLoss'
    }
    
    rename_dict = rename_dict_english

    df = df.rename(columns=rename_dict)
    elixhauser_category_columns_cleaned = [rename_dict.get(col) for col in elixhauser_category_columns_cleaned]

    return df, elixhauser_category_columns_cleaned

df_elixhauser, elixhauser_category_columns = clean_names(df_elixhauser)

## Exclusion

In [ ]:
len(df_elixhauser)

In [ ]:
df_elixhauser['asa_status'].value_counts()

In [ ]:
"""
import operator
df_elixhauser = utils.exclude_rows(df_elixhauser, 'asa_status', [4.0], operator.gt) # NOTE: will also remove NaNs 
df_elixhauser.reset_index(inplace=True)
"""

In [ ]:
len(df_elixhauser)

## EDA

In [ ]:
# explore data quality
utils.get_eda_metrics(df_elixhauser)

## Split

In [94]:
# splot data for analysis
X = df_elixhauser[elixhauser_category_columns].copy()
Y = df_elixhauser[elixhauser_outcome_column].copy()

In [ ]:
X.shape

In [ ]:
Y.shape

# Grid Search (LCA Models)

## Search

In [282]:
def run_gs(grid):
    # base model
    model = StepMix(n_components=3, n_steps=1, measurement='bernoulli', structural='bernoulli', random_state=42)

    gs = GridSearchCV(estimator=model, cv=3, param_grid=grid)
    gs.fit(X, Y)

    return gs

In [ ]:
gs = run_gs(grid)

## Evaluation

In [284]:
def evaluate_gs(gs, grid):

    # log likelihood
    results = pd.DataFrame(gs.cv_results_)
    results['Val. Log Likelihood'] = results['mean_test_score']
    sns.set_style('darkgrid')
    sns.lineplot(data=results, x='param_n_components', y='Val. Log Likelihood', hue='param_n_steps', palette='Dark2')

    # other metrics (run all models (again) and save parameters)

    bic_values = []
    sabic_values = []
    entropy_values = []
    ns_patients = []

    for i in range(len(results['params'])):

        params = results['params'][i]
        current_model = gs.estimator.set_params(**params)
        current_model.fit(X,Y)
        
        bic = current_model.bic(X, Y)
        sabic = current_model.sabic(X, Y)
        entropy = current_model.relative_entropy(X, Y)
        
        bic_values.append(bic)
        sabic_values.append(sabic)
        entropy_values.append(entropy)

        # smallest class (n_patients)
        array_predictios = current_model.predict(X,Y)
        df_predictions = pd.DataFrame(array_predictios, columns=['class'])
        assigned_classes_patients = pd.merge(df_elixhauser, df_predictions, left_index=True, right_index=True)
        # assigned_classes_patients = get_assigned_classes_patients(current_model)
        patient_counts = assigned_classes_patients['class'].value_counts().to_dict()

        if patient_counts != {}:
            smallest_class = min(patient_counts, key=patient_counts.get)
            ns_patients.append(patient_counts.get(smallest_class))
        else:
            print(f'Skipping smallest_class for {i}')

        # smallest probability
        # array_proba = current_model.predict_proba(X,Y)

    plt.figure(figsize=(12, 6))
    plt.plot(grid['n_components'], bic_values)
    plt.xticks(grid['n_components'])
    plt.xlabel('Number of Latent Classes')
    plt.ylabel('BIC')
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(grid['n_components'], sabic_values)
    plt.xticks(grid['n_components'])
    plt.xlabel('Number of Latent Classes')
    plt.ylabel('SABIC')
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(grid['n_components'], entropy_values)
    plt.xticks(grid['n_components'])
    plt.xlabel('Number of Latent Classes')
    plt.ylabel('Scaled Relative Entropy')
    plt.grid(True)
    plt.show()

    if len(ns_patients) != 0:
        plt.figure(figsize=(12, 6))
        plt.plot(grid['n_components'], ns_patients)
        plt.xticks(grid['n_components'])
        plt.xlabel('Number of Latent Classes')
        plt.ylabel('Cases in Smallest Class')
        plt.grid(True)
        plt.show()
    else:
        print(f'Skipping ns_patients for {i}')

    df_summary = pd.DataFrame({
        'Number of Latent Classes': grid['n_components'],
        'BIC': bic_values,
        'SABIC': sabic_values,
        'Relative Entropy': entropy_values,
        'Cases in Smallest Class': ns_patients
    })

    return df_summary

In [ ]:
df_summary = evaluate_gs(gs, grid)
df_summary

In [ ]:
render_mpl_table(df_summary, col_width=4)

# BLRT (LCA Models)

In [287]:
# baseline model
# model = StepMix(n_components=3, n_steps=1, measurement='bernoulli', structural='bernoulli', random_state=42, verbose=0, progress_bar=0)
# p_values = blrt_sweep(model, X, Y, low=grid['n_components'][0], high=grid['n_components'][-1], n_repetitions=2) # n_repetitions for actual analysis

In [288]:
# p_values < .05

# Inspection of Models

## Utilities

### Model

In [97]:
def get_model(n_classes, gs=None):

    # if grid search results are included, extract data from there
    if gs:
        results = pd.DataFrame(gs.cv_results_)
        results['params'][n_classes-1]
        selected_params = results['params'][n_classes-1]
        selected_model = gs.estimator.set_params(**selected_params) 
        
    else:
        selected_model = StepMix(n_components=n_classes, n_steps=1, measurement='bernoulli', structural='bernoulli', random_state=42)
    
    selected_model.fit(X,Y)

    # selected_model.get_params()
    # selected_model.report(X, Y)

    return selected_model

### Probabilities

In [98]:
def get_incidence_probabilities(model):
    array_predictios = model.predict(X,Y)
    df_predictions = pd.DataFrame(array_predictios, columns=['class'])

    df_merged = pd.merge(X, df_predictions, left_index=True, right_index=True)
    df_merged['class'] = df_merged['class'] + 1
    df_probabilities = df_merged.copy().groupby('class')[elixhauser_category_columns].mean().reset_index()
    
    # total prevalence
    incidences = X[elixhauser_category_columns].sum()
    cohort_size = X.shape[0]
    total_prevalence = incidences / cohort_size

    # class incidences
    class_incidences = df_merged.groupby('class')[elixhauser_category_columns].sum()
    class_sizes = df_merged['class'].value_counts()

    df_normalized_probabilities = df_probabilities.copy()

    # class and normalized prevalence
    for column in elixhauser_category_columns:
            class_prevalence = class_incidences[column] / class_sizes
            df_normalized_probabilities[column] = df_probabilities['class'].map(class_prevalence) / total_prevalence[column]
    
    return df_normalized_probabilities

### Assigned Class

In [99]:
def get_assigned_classes_patients(model):
    array_predictios = model.predict(X,Y)
    df_predictions = pd.DataFrame(array_predictios, columns=['class'])
    df_predictions['class'] = df_predictions['class'] + 1 # remove 0 indexing
    df_merged = pd.merge(df_elixhauser, df_predictions, left_index=True, right_index=True)

    return df_merged

In [100]:
def get_multiple_assigned_classes_patients(models_class_numbers:list):
    
    list_predictions_dfs = []

    for n_classes in models_class_numbers:
        model = get_model(n_classes, False)
        array_predictions = model.predict(X,Y)
        df_predictions = pd.DataFrame(array_predictions, columns=[f'class_{n_classes}'])
        df_predictions[f'class_{n_classes}'] = df_predictions[f'class_{n_classes}'] + 1
        list_predictions_dfs.append(df_predictions)

    df_all_predictions = pd.concat(list_predictions_dfs, axis=1)
    df_merged = pd.concat([df_elixhauser, df_all_predictions], axis=1)

    return df_merged

In [101]:
def get_assigned_classes_categories(probabilities):

    category_max_values = {}
    category_max_classes = {}

    for category in elixhauser_category_columns:

        # find max value = assigned class
        max_value = probabilities[category].max()
        max_class = probabilities[probabilities[category] == max_value]['class'].values[0]
        category_max_values[category] = max_value
        category_max_classes[category] = max_class

    df_assigned_class = pd.DataFrame.from_dict({
        'Category': list(category_max_values.keys()),
        'Highest Value': list(category_max_values.values()),
        'Class with Highest Value': [f'{category_max_classes[category]}' for category in category_max_values]
    })

    return df_assigned_class.sort_values('Class with Highest Value')

In [102]:
def get_class_categories(assigned_classes):
    df_class_categories = assigned_classes.groupby('Class with Highest Value')['Category'].apply(list).reset_index()
    df_class_categories['Number of Categories'] = df_class_categories['Category'].apply(len)
    return df_class_categories

### Graph

In [111]:
def get_polar_plot(model, class_order:list):
    fig = go.Figure()
    
    # helpers for class numbers
    probabilities = get_incidence_probabilities(model)
    assigned_classes = get_assigned_classes_categories(probabilities)
    class_categories = get_class_categories(assigned_classes)
    category_count_dict = dict(zip(class_categories['Class with Highest Value'], class_categories['Number of Categories']))

    # helper for patient numbers
    assigned_classes_patients = get_assigned_classes_patients(model)
    patient_counts = assigned_classes_patients['class'].value_counts().to_dict()

    # helper for incidence numbers
    # incidences = X[elixhauser_category_columns].sum()
    # incidence_counts = incidences.sort_values(ascending=False).to_dict()

    # helper for category order (NOTE: specific to amount of classes)
    cat_one = list(class_categories[class_categories['Class with Highest Value'] == '1']['Category'])
    cat_two = list(class_categories[class_categories['Class with Highest Value'] == '2']['Category'])
    cat_three = list(class_categories[class_categories['Class with Highest Value'] == '3']['Category'])
    cat_four = list(class_categories[class_categories['Class with Highest Value'] == '4']['Category'])
    cat_five = list(class_categories[class_categories['Class with Highest Value'] == '5']['Category'])
    ordered_categories = cat_one + cat_two + cat_three + cat_four + cat_five
    ordered_categories = [
        x
        for xs in ordered_categories
        for x in xs
    ]

    # sort classes accordingly
    class_order_dict = {class_id: index for index, class_id in enumerate(class_order)}
    sorted_classes = sorted(probabilities['class'].unique(), key=lambda c: class_order_dict.get(c, float('inf')))
    class_rename_dict = {class_id: f'{index + 1}' for index, class_id in enumerate(class_order)}

    # graph colors (NOTE: specific to amount of classes)

    colors = {
            '1': 'rgba(137, 146, 251, 0.1)', 
            '2': 'rgba(241, 108, 86, 0.1)',    
            '3': 'rgba(114, 219, 197, 0.1)',   
            '4': 'rgba(200, 166, 247, 0.1)',  
            '5': 'rgba(251, 170, 109, 0.1)'    
    }

    line_colors = {
            '1': 'rgba(137, 146, 251, 1)', 
            '2': 'rgba(241, 108, 86, 1)',    
            '3': 'rgba(114, 219, 197, 1)',   
            '4': 'rgba(200, 166, 247, 1)',  
            '5': 'rgba(251, 170, 109, 1)'    
    }


    # iterate over each class
    max_value = 0
    for class_label in sorted_classes:
        # get class data
        class_data = probabilities[probabilities['class'] == class_label]
        values = class_data[ordered_categories].values.flatten() 
        categories = ordered_categories
        max_value = max(max_value, max(values))

        # helper for class numbers
        n_categories = category_count_dict.get(str(class_label), 0)

        # helper for patient numbers
        n_patients = patient_counts.get(class_label, 0)

        # helper for new category labels
        # category_labels_with_incidence = [f"{cat} (cases: {incidence_counts.get(cat, 0)})" for cat in categories]

        renamed_class = class_rename_dict.get(class_label, class_label)

        fill_color = colors.get(renamed_class) 
        line_color = line_colors.get(renamed_class) 

        # add data to graph
        fig.add_trace(go.Scatterpolar(
            r=values.tolist() + [values[0]],
            theta=categories + [categories[0]],
            name=f'Cluster {renamed_class} (categories: {n_categories} | cases: {n_patients})',
            fill='toself', 
            fillcolor=fill_color,
            line=dict(color=line_color)
        ))

    # add line at prevalence ? 3
    num_categories = len(ordered_categories)
    circle_r = [3] * num_categories
    fig.add_trace(go.Scatterpolar(
        r=circle_r + [circle_r[0]], 
        theta=ordered_categories + [ordered_categories[0]], 
        mode='lines',
        line=dict(color='red', dash='dash'), 
        name='Prevalence = 3',
        showlegend=True
    ))

    # adjust layout
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                    visible=True,
                    showline=True, 
                    linecolor='rgba(0,0,0,0.1)',
                    gridcolor='rgba(0,0,0,0.1)',
                    range=[0, max_value]
                ),
            angularaxis=dict(
                tickfont=dict(
                    size=24),
                linecolor='grey',
                gridcolor='rgba(0,0,0,0.1)'
            ),
            bgcolor='white'
        ),
        width=2400,
        height=1200,
        showlegend=True,
        legend=dict(
            font=dict(
                size=24
            )
        ),
        paper_bgcolor='rgba(255,255,255)',
        plot_bgcolor='rgba(255,255,255)'
    )

    return fig

### Pipeline

In [104]:
def run_pipeline(n_classes):
    selected_model = get_model(n_classes)
    fig = get_polar_plot(selected_model)
    probabilities = get_incidence_probabilities(selected_model)
    category_classes = get_assigned_classes_categories(probabilities)
    patient_classes = get_assigned_classes_patients(selected_model)
    class_categories = get_class_categories(category_classes)

    return fig, category_classes, class_categories, patient_classes

## Models

In [ ]:
selected_model = get_model(5)

In [ ]:
probabilities = get_incidence_probabilities(selected_model)
probabilities

In [ ]:
category_classes = get_assigned_classes_categories(probabilities)
category_classes

In [ ]:
patient_classes_5 = get_assigned_classes_patients(selected_model)
class_categories = get_class_categories(category_classes)
class_categories

In [109]:
class_order = [5, 3, 4, 1, 2]

In [ ]:
fig = get_polar_plot(selected_model, class_order)
fig

# Table 1

In [310]:
df_merged = pd.merge(df_elixhauser, patient_classes_5[['case_id', 'class']], on='case_id', how='left')

In [ ]:
df_merged, _ = clean_names(df_merged)
rename_dict = {
    'age_during_op': 'Alter bei OP', 
    'female_sex': 'Geschlecht weiblich', 
    'asa_status': 'ASA Status',
    'elixhauser_van_walraven': 'Elixhauser',
    'bmi': 'BMI'
    }
df_merged = df_merged.rename(columns=rename_dict)
df_merged.columns

In [ ]:
df_cleaned = df_merged.drop(columns=['pat_id', 'case_id', 'op_length', 'weight', 'height', 'admission_date_time', 'discharge_date_time', 'ops_code', 'op_date_time', 'birth_date', 'op_start_date_time', 'op_end_date_time', 'elixhauser_AHRQ', 'class', 'elevated_risk_surgery', 'MI_history', 'CD_history', 'prior_insulin', 'prior_creatinine', 'vascular_disease_history', 'STT_history', 'AF_history', 'in_hospital_death', 'stroke_30_days', 'MACE_30_days', 'RCRI', 'CHA2DS2_VASc'])
df_cleaned.columns

In [ ]:
# order classes
df_merged['class'] = df_merged['class'].astype(str)
class_order_dict = {str(class_id): index + 1 for index, class_id in enumerate(class_order)}
df_merged['class'] = df_merged['class'].map(lambda x: class_order_dict.get(x))
df_merged

In [ ]:
import pandas as pd
from tableone import TableOne

data = df_merged

columns = df_cleaned.columns.tolist()
categorical = df_cleaned.drop(columns=columns).columns.tolist()

groupby = 'class'

mytable = TableOne(data=data, columns=columns, categorical=categorical, groupby=groupby, pval=True)

In [ ]:
df_merged

In [ ]:
# view as DF

mytable.to_csv('data/subphenotypes/table-one.csv')
df_tableOne = pd.read_csv('data/subphenotypes/table-one.csv', delimiter = ',')

tableOne_columns = [
    '', 
    '', 
    'Fehlende',
    'Gesamt', 
    '1',
    '2',
    '3',
    '4',
    '5',
    'P-Wert'
]

df_tableOne.columns = tableOne_columns

df_tableOne = df_tableOne.drop(index=0)
df_tableOne = df_tableOne.reset_index(drop=True)
df_tableOne.to_csv('data/subphenotypes/table-one_adj.csv')

render_mpl_table(df_tableOne, col_width=5.5)

In [ ]:
print(mytable.tabulate(tablefmt='github'))

# References

- https://stepmix.readthedocs.io/en/latest/index.html
- https://colab.research.google.com/drive/1btXHCx90eCsnUlQv_yN-9AzKDhJP_JkG?usp=drive_link#scrollTo=132vQNd8wE3J